# Transmisor

In [12]:
import numpy as np
from PIL import Image

# Filtro RRC (si decides mantener este aspecto)
def rrc_filter(beta, sps, num_taps):
    t = np.arange(-num_taps // 2, num_taps // 2 + 1) / sps
    pi_t = np.pi * t
    four_beta_t = 4 * beta * t

    with np.errstate(divide='ignore', invalid='ignore'):
        numerator = np.sin(pi_t * (1 - beta)) + 4 * beta * t * np.cos(pi_t * (1 + beta))
        denominator = pi_t * (1 - (four_beta_t) ** 2)
        h = numerator / denominator

    h[np.isnan(h)] = 1.0 - beta + (4 * beta / np.pi)
    t_special = np.abs(t) == (1 / (4 * beta))
    h[t_special] = (beta / np.sqrt(2)) * (
        ((1 + 2 / np.pi) * np.sin(np.pi / (4 * beta))) +
        ((1 - 2 / np.pi) * np.cos(np.pi / (4 * beta)))
    )
    h /= np.sqrt(np.sum(h ** 2))

    return h

# Convertir imagen en una secuencia de bits
def image_to_bits(image_path):
    img = Image.open(image_path)
    img_array = np.array(img)
    img_bits = np.unpackbits(img_array)
    return img_bits, img_array.shape

# Codificar cabecera (tamaño de la imagen)
def encode_header(image_shape):
    height, width, channels = image_shape
    header = f'{height:016b}' + f'{width:016b}' + f'{channels:08b}'  # Codificación en bits
    return np.array([int(bit) for bit in header], dtype=np.uint8)

# Convertir bits a símbolos BPSK
def bits_to_symbols(bits):
    return 2 * bits - 1

# Generar el preámbulo (puede ser una secuencia conocida)
def generate_preamble(length=64):
    return np.random.randint(0, 2, size=length)  # Secuencia aleatoria de bits para sincronización

# Crear señal completa con cabecera, preámbulo y datos
def create_signal(image_path):
    # Convertir imagen a bits
    img_bits, img_shape = image_to_bits(image_path)
    
    # Generar cabecera
    header_bits = encode_header(img_shape)
    
    # Generar preámbulo
    preamble_bits = generate_preamble()
    
    # Concatenar preámbulo, cabecera y bits de la imagen
    full_signal_bits = np.concatenate([preamble_bits, header_bits, img_bits])
    
    # Convertir bits a símbolos BPSK
    signal_symbols = bits_to_symbols(full_signal_bits)
    
    return signal_symbols, preamble_bits, header_bits, img_shape

# Simular la transmisión
def transmit_signal(image_path):
    signal_symbols, preamble_bits, header_bits, img_shape = create_signal(image_path)
    print(f"Transmitiendo señal: Preámbulo, Cabecera (Tamaño de la imagen: {img_shape}), y Bits de la imagen.")
    return signal_symbols

# Simulación del transmisor
image_path = 'Escudo_UNSA.png'
transmitted_signal = transmit_signal(image_path)


Transmitiendo señal: Preámbulo, Cabecera (Tamaño de la imagen: (599, 488, 4)), y Bits de la imagen.


# Receptor

In [13]:
# Filtro RRC
def rrc_filter(beta, sps, num_taps):
    t = np.arange(-num_taps // 2, num_taps // 2 + 1) / sps
    pi_t = np.pi * t
    four_beta_t = 4 * beta * t

    with np.errstate(divide='ignore', invalid='ignore'):
        numerator = np.sin(pi_t * (1 - beta)) + 4 * beta * t * np.cos(pi_t * (1 + beta))
        denominator = pi_t * (1 - (four_beta_t) ** 2)
        h = numerator / denominator

    h[np.isnan(h)] = 1.0 - beta + (4 * beta / np.pi)
    t_special = np.abs(t) == (1 / (4 * beta))
    h[t_special] = (beta / np.sqrt(2)) * (
        ((1 + 2 / np.pi) * np.sin(np.pi / (4 * beta))) +
        ((1 - 2 / np.pi) * np.cos(np.pi / (4 * beta)))
    )
    h /= np.sqrt(np.sum(h ** 2))

    return h

# Convertir símbolos BPSK a bits
def symbols_to_bits(symbols):
    bits = (symbols + 1) // 2
    return np.round(bits).astype(np.int32)  # Asegurarse de que los bits sean enteros

# Decodificar la cabecera para obtener el tamaño de la imagen
def decode_header(header_bits):
    height = int("".join(map(str, header_bits[:16])), 2)
    width = int("".join(map(str, header_bits[16:32])), 2)
    channels = int("".join(map(str, header_bits[32:40])), 2)
    return height, width, channels

# Convertir bits a imagen, verificando la forma de la imagen
def bits_to_image(bits, image_shape):
    total_pixels = np.prod(image_shape)
    total_bits_needed = total_pixels * 8  # Necesitamos tantos bits como píxeles en la imagen
    print(f"Total bits necesarios: {total_bits_needed}")
    print(f"Total bits recibidos: {len(bits)}")

    if len(bits) < total_bits_needed:
        raise ValueError(f"Insuficientes bits recibidos para reconstruir la imagen. "
                         f"Esperados: {total_bits_needed}, Recibidos: {len(bits)}")
    
    bits = bits[:total_bits_needed]  # Recortar a los bits necesarios

    # Empaquetar bits de nuevo a valores de píxeles
    img_array = np.packbits(bits)
    
    # Verificar si los datos se ajustan al formato esperado
    if len(img_array) != total_pixels:
        raise ValueError(f"El tamaño de los datos no coincide con la forma esperada de la imagen. "
                         f"Esperado: {total_pixels}, obtenido: {len(img_array)}")

    # Asegurarse de que la forma es correcta para la imagen
    img_array = img_array.reshape(image_shape)
    
    # Crear la imagen usando PIL
    if image_shape[2] == 1:  # Si es una imagen en blanco y negro
        img = Image.fromarray(img_array[:, :, 0], mode='L')  # Modo 'L' para escala de grises
    else:
        img = Image.fromarray(img_array.astype(np.uint8), mode='RGB')  # Imagen RGB
    return img

# Recepción de señal simulada
def receive_signal(signal_symbols, preamble_len=64, header_len=40, samples_per_symbol=8, beta=0.2, num_taps=151):
    # Aplicar el filtro RRC
    rrc_coef = rrc_filter(beta, samples_per_symbol, num_taps)
    received_filtered = np.convolve(signal_symbols, rrc_coef, mode='same')
    
    # Demodular símbolos a bits
    received_symbols = np.sign(np.real(received_filtered))
    received_bits = symbols_to_bits(received_symbols)
    
    # Extraer preámbulo, cabecera y datos
    preamble_bits = received_bits[:preamble_len]
    header_bits = received_bits[preamble_len:preamble_len + header_len]
    img_bits = received_bits[preamble_len + header_len:]
    
    # Decodificar cabecera
    img_shape = decode_header(header_bits)
    print(f"Cabecera decodificada: {img_shape}")
    print(f"Preámbulo decodificado: {preamble_bits}")
    print(f"Total bits recibidos para la imagen: {len(img_bits)}")

    # Reconstruir imagen
    reconstructed_img = bits_to_image(img_bits, img_shape)
    
    return reconstructed_img, img_shape

# Simulación de la recepción
def simulate_reception(transmitted_signal, sps, beta):
    reconstructed_img, img_shape = receive_signal(transmitted_signal, samples_per_symbol=sps, beta=beta)
    print(f"Recibida imagen de tamaño: {img_shape}")
    reconstructed_img.show()
    reconstructed_img.save('reconstructed_image.png')

# Ejecutar la simulación con la señal transmitida
simulate_reception(transmitted_signal, sps=8, beta=0.2)


Cabecera decodificada: (28, 56, 0)
Preámbulo decodificado: [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0]
Total bits recibidos para la imagen: 9353984
Total bits necesarios: 0
Total bits recibidos: 9353984


ValueError: not enough image data